In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!pip install geoplot
!pip install geodatasets
!pip install libpysal
!pip install esda
!pip install pysal
!pip install mapclassify -q
!pip install pysal -q
!pip install pydeck
!pip install selenium
# Chrome 설치
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install chromedriver-autoinstaller

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import geoplot as gplt
import geoplot.crs as gcrs
import seaborn as sns
import contextily as ctx
from geodatasets import get_path
import geopandas as gpd
import imageio
from pathlib import Path
import imageio.v2 as imageio
from libpysal import weights
from esda import Moran
from sklearn.neighbors import KernelDensity
from mapclassify import NaturalBreaks
from esda import Moran_Local
from pysal.lib import weights
import folium
import libpysal as lps
import esda
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
import pydeck as pdk
from pyproj import Transformer
import re
from IPython.core.display import display, HTML
from IPython.display import IFrame
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service
import sys

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import seaborn as sns

# 폰트 경로 가져오기
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 폰트 설정
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 설치된 폰트 확인 (선택 사항)
[f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]

['NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic']

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DATA

In [ ]:
od_final = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_final.csv')

In [ ]:
sido_geojson = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/sido.geojson")

In [ ]:
od_final.drop('Unnamed: 0',axis=1,inplace=True)
od_final.head()

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,...,시군구명,읍면동명,동리명,축제 지역,dest_위도,dest_경도,origin_지역,위도,경도,dest_지역
0,4420059000,4420060000,2023-09-01,08:00,08:00,1,4,1.0,0.0,1,...,아산시,온양4동,방축동,충청남도 아산시 온양4동,36.783522,127.012897,충청남도 아산시 온양3동,36.798244,126.978442,충청남도 아산시 온양4동
1,4420059000,4420060000,2023-09-01,08:00,08:00,1,4,1.0,0.0,1,...,아산시,온양4동,방축동,충청남도 아산시 온양4동,36.783522,127.012897,충청남도 아산시 온양3동,36.798244,126.978442,충청남도 아산시 온양4동
2,4420059000,4420060000,2023-09-01,08:00,08:00,1,4,1.0,0.0,1,...,아산시,온양4동,방축동,충청남도 아산시 온양4동,36.783522,127.012897,충청남도 아산시 온양3동,36.798244,126.978442,충청남도 아산시 온양4동
3,2717058000,2717064000,2023-09-01,16:00,17:00,0,1,0.0,0.0,4,...,서구,평리3동,평리동,대구광역시 서구 평리3동,35.869754,128.574285,대구광역시 서구 비산4동,35.875918,128.561310,대구광역시 서구 평리3동
4,4817072000,4817073000,2023-09-01,16:00,17:00,1,4,0.0,0.0,4,...,진주시,판문동,판문동,경상남도 진주시 판문동,35.192831,128.061707,경상남도 진주시 이현동,35.187656,128.039054,경상남도 진주시 판문동


In [ ]:
coord_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/coordinate_UTMK_이름포함.tsv', header=0, sep='\t')
coord_df['ADMCD'] = coord_df['ADMCD'].astype('str')
coord_df.head()

,SERIAL,ADMCD,ADMNM,X,Y
0,0,1100000000,서울특별시,954919,1950195
1,1,1111000000,서울특별시 종로구,953003,1955642
2,2,1111051000,서울특별시 종로구 청운동,953064,1954062
3,3,1111051500,서울특별시 종로구 청운효자동,953064,1954062
4,4,1111052000,서울특별시 종로구 효자동,953064,1954062


In [ ]:
# UTMK에서 WGS84로 좌표 변환
transformer = Transformer.from_crs("EPSG:5179", "EPSG:4326", always_xy=True)
coord_df['lon'], coord_df['lat'] = transformer.transform(coord_df['X'].values, coord_df['Y'].values)
coord_df.head()

,SERIAL,ADMCD,ADMNM,X,Y,lon,lat
0,0,1100000000,서울특별시,954919,1950195,126.989638,37.549992
1,1,1111000000,서울특별시 종로구,953003,1955642,126.967598,37.598991
2,2,1111051000,서울특별시 종로구 청운동,953064,1954062,126.968391,37.584754
3,3,1111051500,서울특별시 종로구 청운효자동,953064,1954062,126.968391,37.584754
4,4,1111052000,서울특별시 종로구 효자동,953064,1954062,126.968391,37.584754


In [ ]:
#정읍 구절초 축제
filtered_df = od_final[(od_final['origin_지역'].str.contains("안양")) | (od_final['dest_지역'].str.contains("안양"))] #축제에 맞게 "시"넣어주시면 됩니다.
filtered_df.head()

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,...,시군구명,읍면동명,동리명,축제 지역,dest_위도,dest_경도,origin_지역,위도,경도,dest_지역
21,4117359000,4129051000,2023-09-01,08:00,08:00,1,2,2.0,0.0,1,...,과천시,중앙동,중앙동,경기도 과천시 중앙동,37.380641,126.953269,경기도 안양시 동안구 호계2동,37.433151,126.993456,경기도 과천시 중앙동
45,4117357600,4117357000,2023-09-01,14:00,15:00,0,0,0.0,0.0,0,...,안양시 동안구,평촌동,평촌동,경기도 안양시 동안구 평촌동,37.390394,126.964248,경기도 안양시 동안구 평안동,37.394650,126.977075,경기도 안양시 동안구 평촌동
59,4117357600,4117357000,2023-09-01,21:00,22:00,0,0,4.0,0.0,0,...,안양시 동안구,평촌동,평촌동,경기도 안양시 동안구 평촌동,37.390394,126.964248,경기도 안양시 동안구 평안동,37.394650,126.977075,경기도 안양시 동안구 평촌동
85,4117356000,4117357000,2023-09-01,09:00,09:00,0,5,4.0,1.0,1,...,안양시 동안구,평촌동,평촌동,경기도 안양시 동안구 평촌동,37.402271,126.970987,경기도 안양시 동안구 관양2동,37.394650,126.977075,경기도 안양시 동안구 평촌동
297,4117356000,4117357000,2023-09-01,19:00,19:00,1,5,4.0,0.0,0,...,안양시 동안구,평촌동,평촌동,경기도 안양시 동안구 평촌동,37.402271,126.970987,경기도 안양시 동안구 관양2동,37.394650,126.977075,경기도 안양시 동안구 평촌동


In [ ]:
# 저장할 폴더 경로
save_folder = "/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result_안양"  # 원하는 경로로 변경하세요.

# 폴더가 존재하지 않으면 생성
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Headless 모드 활성화
chrome_options.add_argument("--no-sandbox")  # 샌드박스 비활성화
chrome_options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화 (리소스 부족 문제 방지)
chrome_options.add_experimental_option("detach", True)

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(3)  # 암묵적 대기 시간 설정

In [ ]:
#driver 관련 오류가 있을 수 있음
#1. 코드 실행 이후 중단하면 driver quit되어 있는 상태라 chrome_options~driver 지정까지의 코드를 한번 더 실행해주셔야 합니다.
print(driver) #여기에 아무 말이 안나온다면.. driver관련 문제를 직접 해결해야합니다.

<selenium.webdriver.chrome.webdriver.WebDriver (session="62d546a19c8faf117974cf1ab58fe382")>


In [ ]:
filtered_df[filtered_df['축제명']=="보은대추축제"] #기준이 되는 축제명과 head하셔도 되는데 한 행만 있어야 다른 정보들 보기 편해서 적절히 조절하세요.

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,...,시군구명,읍면동명,동리명,축제 지역,dest_위도,dest_경도,origin_지역,위도,경도,dest_지역
176,4372032500,4372025000,2023-09-01,19:00,20:00,0,0,0.0,2.0,0,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.469742,127.788715,충청북도 보은군 장안면,36.483639,127.717071,충청북도 보은군 보은읍
931,4372036000,4372025000,2023-09-01,12:00,12:00,0,5,0.0,4.0,0,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.479411,127.698723,충청북도 보은군 수한면,36.483639,127.717071,충청북도 보은군 보은읍
1128,4833031000,4372025000,2023-09-01,18:00,23:00,0,0,0.0,4.0,5,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,35.323520,129.061861,경상남도 양산시 동면,36.483639,127.717071,충청북도 보은군 보은읍
2763,4146559000,4372025000,2023-09-01,14:00,17:00,0,0,0.0,0.0,5,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,37.316488,127.069042,경기도 용인시 수지구 성복동,36.483639,127.717071,충청북도 보은군 보은읍
2799,4372035000,4372025000,2023-09-01,14:00,14:00,0,1,0.0,4.0,4,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.395793,127.727606,충청북도 보은군 삼승면,36.483639,127.717071,충청북도 보은군 보은읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017301,4372031500,4372025000,2023-10-15,11:00,11:00,1,3,1.0,5.0,5,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.512116,127.810351,충청북도 보은군 속리산면,36.483639,127.717071,충청북도 보은군 보은읍
5017385,4372039000,4372025000,2023-10-15,13:00,14:00,1,5,1.0,4.0,0,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.577920,127.664058,충청북도 보은군 내북면,36.483639,127.717071,충청북도 보은군 보은읍
5017696,4311133000,4372025000,2023-10-15,13:00,14:00,1,5,0.0,4.0,3,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.553570,127.548828,충청북도 청주시 상당구 가덕면,36.483639,127.717071,충청북도 보은군 보은읍
5017704,4372036000,4372025000,2023-10-15,13:00,14:00,1,5,0.0,5.0,5,...,보은군,보은읍,이평리,충청북도 보은군 보은읍,36.479411,127.698723,충청북도 보은군 수한면,36.483639,127.717071,충청북도 보은군 보은읍


In [ ]:
sido_geojson

,sido_cd,sido_nm,geometry
0,48,경상남도,"MULTIPOLYGON (((128.04827 34.69187, 128.04939 ..."
1,26,부산광역시,"MULTIPOLYGON (((128.97182 35.03469, 128.97226 ..."
2,43,충청북도,"MULTIPOLYGON (((127.47144 36.47447, 127.47093 ..."
3,27,대구광역시,"MULTIPOLYGON (((128.53241 35.70549, 128.53248 ..."
4,44,충청남도,"MULTIPOLYGON (((126.05093 36.19743, 126.05063 ..."
5,11,서울특별시,"MULTIPOLYGON (((126.91236 37.43859, 126.91135 ..."
6,28,인천광역시,"MULTIPOLYGON (((125.99454 37.03567, 125.99455 ..."
7,45,전라북도,"MULTIPOLYGON (((126.24899 35.57763, 126.24915 ..."
8,46,전라남도,"MULTIPOLYGON (((125.12399 34.08583, 125.12194 ..."
9,30,대전광역시,"MULTIPOLYGON (((127.35975 36.26291, 127.35966 ..."


In [ ]:
# 필터링하여 정읍 부분만 남기기
jeongeup_geojson = sido_geojson[sido_geojson['sido_nm'] == '경기도'] #시도 데이터여서 특별자치시가 아니면 시 데이터는 없어

In [ ]:
filtered_df[filterd_df['date'] > '2023-09-28']

# text 있는 version

바꿔야하는 부분

**1. 축제의 시작 및 종료 날짜 부분, 정읍 지역을 표시하는 ~ 부분 (필수)**
2. text 위치가 마음에 안든다면 ~추가 부분에서 위도, 경도 부분을 변경하면서 조정가능(조정)

In [ ]:
for (date, end_time), group in filtered_df.groupby(['date', 'end_time']):
    # 그룹 내에서 데이터 필터링
    df_subset = group

    # Define origin and destination arrow colors
    origin_color = [255, 0, 255, 255]  # 아산시에서 출발하는 화살표 색상 (magenta)
    dest_color = [0, 0, 255, 255]  # 아산시로 도착하는 화살표 색상 (blue)

    # 축제의 시작 및 종료 날짜
    festival_start_date = "2023-10-05"  # 여기에 실제 시작 날짜 입력 -> filtered_df기준으로 넣어줫습니다.
    festival_end_date = "2023-10-15"  # 여기에 실제 종료 날짜 입력
    festival_name = "제16회 정읍 구절초꽃축제"  # 여기에 축제 이름 입력
    festival_latitude = 35.610193  # 축제 위도 filtered_df에서 그냥 위도, 경도라고 써진 열 기준 값을 넣으면 됩니다.
    festival_longitude = 126.907693  # 축제 경도

    # Pydeck layer 설정
    layers = [
        pdk.Layer(
            'ArcLayer',
            df_subset,
            get_source_position='[origin_경도, origin_위도]',
            get_target_position='[dest_경도, dest_위도]',
            get_width='3 + 20 * 정규화_이용',  # 정규화된 이용량에 따른 선 너비
            get_source_color=origin_color,  # 출발지 색상
            get_target_color=dest_color,  # 도착지 색상
            pickable=True,
            auto_highlight=True
        ),
        # 정읍 지역을 표시하는 GeoJsonLayer 추가
        pdk.Layer(
            "GeoJsonLayer",
            jeongeup_geojson,  # 정읍 지역만 포함된 GeoJSON 데이터 -> 앞에서 설정한 jeongeup_geojson이름만 바꿔주면 됩니다.
            get_fill_color='[255, 255, 255, 128]',  # 흰색으로 채움 (반투명)
            get_line_color='[255, 0, 0, 255]',  # 빨간색 테두리
            line_width_min_pixels=2,
            pickable=True,
        ),
        # 시점 제목 추가
        pdk.Layer(
            'TextLayer',
            pd.DataFrame({
                'text': [f'시점: {date} {end_time}'],
                'lat': [35.6],  # 원하는 위치 -> 위도 기준으로 들어가는거라 조정하면서 해야할드슈ㅠ
                'lon': [125.5],
            }),
            get_position='[lon, lat]',
            get_text_color='[255, 255, 255,255]',  # 흰색
            get_size=8,  # 작게 표시
            get_text_anchor="'middle'",  # 중앙 정렬
            pickable=False,
        ),
        # 축제명 추가
        pdk.Layer(
            'TextLayer',
            pd.DataFrame({
                'text': [f'{festival_name}'],
                'lat': [35.5],  # 원하는 위도
                'lon': [125.5],  # 원하는 경도
            }),
            get_position='[lon, lat]',
            get_text_color='[255, 255, 255,255]',  # 흰색
            get_size=8,  # 작게 표시
            get_text_anchor="'middle'",  # 중앙 정렬
            pickable=False,
        ),
        # 축제 기간 추가
        pdk.Layer(
            'TextLayer',
            pd.DataFrame({
                'text': [f'기간: {festival_start_date} ~ {festival_end_date}'],
                'lat': [37],  # 축제명 아래쪽으로 위치 조정
                'lon': [125.5],
            }),
            get_position='[lon, lat]',
            get_text_color='[255, 255, 255,255]',  # 흰색
            get_size=10,  # 작게 표시
            get_text_anchor="'middle'",  # 중앙 정렬
            pickable=False,
        ),
    ]

    # Pydeck 뷰 상태 설정
    view_state = pdk.data_utils.compute_view(df_subset[['origin_경도', 'origin_위도']].values)
    view_state.latitude = 35.610193  # 한국 중앙 위도
    view_state.longitude = 126.907693  # 한국 중앙 경도
    view_state.zoom = 5.0  # 적절한 줌 레벨
    view_state.bearing = 0
    view_state.pitch = 0

    # Pydeck 차트 생성
    r = pdk.Deck(layers=layers, initial_view_state=view_state)

    # HTML 파일로 저장
    html_filename = os.path.join(save_folder, f"graph_{date}_{end_time}.html")
    r.to_html(html_filename)

    # PNG 저장을 위한 Chrome 드라이버 설정
    driver.get(f"file://{html_filename}")  # HTML 파일 열기
    time.sleep(3)  # 페이지 로딩 대기

    # PNG 파일 이름 생성 및 저장
    screenshot_filename = os.path.join(save_folder, f"graph_{date}_{end_time}.png")
    driver.save_screenshot(screenshot_filename)

# 드라이버 종료 -> 이후 계속 driver 지정해줘야함. 위코드 돌리러가면 됨.ㅎㅎ
driver.quit()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

이 부분은 변경할 부분이 없습니다. 경로 정도?

In [ ]:
import imageio
from PIL import Image

# 이미지 파일 경로 설정
image_folder = '/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result원주/'
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

# 파일명에서 datetime 추출하여 정렬
image_files.sort(key=lambda x: x.split('_')[-1].replace('.png', ''))  # '_dt' 부분에서 datetime 문자열로 정렬

# 이미지 크기 설정 (원하는 크기로 변경)
desired_size = (800, 800)  # 예: 800x800 픽셀

# 이미지 리스트의 각 이미지를 원하는 크기로 조정
resized_images = []
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)  # 전체 경로 생성
    image = Image.open(img_path)  # PIL로 이미지 열기
    resized_image = image.resize(desired_size, Image.LANCZOS)  # 이미지 크기 조정
    resized_images.append(resized_image)

# GIF 생성
gif_path = '/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result2/jefestival_od.gif'
imageio.mimsave(gif_path, resized_images, fps=3, loop=True)
print(f"GIF 파일이 생성되었습니다: {gif_path}")

# text가 별로다 : text 없는 version

바꿔야하는 부분

**1. 축제의 시작 및 종료 날짜 부분 (필수)**
**2. 정읍을 표시하는 ~ 추가 부분 (필수)**

In [ ]:
for (date, end_time), group in filtered_df.groupby(['date', 'end_time']):
    # 그룹 내에서 데이터 필터링
    df_subset = group

    # Define origin and destination arrow colors
    origin_color = [255, 0, 255, 255]  # 아산시에서 출발하는 화살표 색상 (magenta)
    dest_color = [0, 0, 255, 255]  # 아산시로 도착하는 화살표 색상 (blue)

    # 지역명 데이터를 포함하는 텍스트 레이어 추가
    region_data = pd.DataFrame({
        'lat': df_subset['origin_위도'],  # 출발지 위도
        'lon': df_subset['origin_경도'],  # 출발지 경도
        'text': df_subset['읍면동명'] + ', ' + df_subset['시군구명']  # 지역명 (읍면동 + 시군구)
    })

    # Pydeck layer 설정
    layers = [
        pdk.Layer(
            'ArcLayer',
            df_subset,
            get_source_position='[origin_경도, origin_위도]',
            get_target_position='[dest_경도, dest_위도]',
            get_width='3 + 20 * 정규화_이용',  # 선을 더 굵게 설정
            get_source_color=origin_color,  # 출발지 색상
            get_target_color=dest_color,  # 도착지 색상
            pickable=True,
            auto_highlight=True
        ),
        # 정읍 지역을 표시하는 GeoJsonLayer 추가
        pdk.Layer(
            "GeoJsonLayer",
            jeongeup_geojson,  # 정읍 지역만 포함된 GeoJSON 데이터 -> 앞에서 설정한 jeongeup_geojson이름만 바꿔주면 됩니다.
            get_fill_color='[255, 255, 255, 128]',  # 흰색으로 채움 (반투명)
            get_line_color='[255, 0, 0, 255]',  # 빨간색 테두리
            line_width_min_pixels=2,
            pickable=True,
        ),
        # 지역명 텍스트 레이어 추가
        pdk.Layer(
            'TextLayer',
            region_data,
            get_position='[lon, lat]',
            get_text='text',  # 지역명 텍스트
            get_text_color='[0, 255, 0, 255]',  # 녹색으로 지역명 표시
            get_size=18,
            get_text_anchor="'middle'",
            get_alignment_baseline="'bottom'",
            get_text_font_family="'NanumGothic'",  # 한글 폰트
            pickable=False,
        ),
    ]

    # Pydeck 뷰 상태 설정 (Mapbox 사용)
    view_state = pdk.ViewState(
        latitude=36.5,  # 한국의 중심 위도
        longitude=127.5,  # 한국의 중심 경도
        zoom=5.0,  # 더 넓은 범위를 볼 수 있도록 줌 아웃
        bearing=0,
        pitch=0
    )

    # Mapbox 스타일 추가 (지역명과 지형 정보를 포함하는 배경 타일)
    r = pdk.Deck(
        layers=layers,
        initial_view_state=view_state
    )

    # HTML 파일로 저장
    html_filename = os.path.join(save_folder, f"graph_{date}_{end_time}.html")
    r.to_html(html_filename)

    # PNG 저장을 위한 Chrome 드라이버 설정
    driver.get(f"file://{html_filename}")  # HTML 파일 열기
    time.sleep(3)  # 페이지 로딩 대기

    # PNG 파일 이름 생성 및 저장
    screenshot_filename = os.path.join(save_folder, f"graph_{date}_{end_time}.png")
    driver.save_screenshot(screenshot_filename)

# 드라이버 종료
driver.quit()

In [ ]:
import imageio
from PIL import Image

# 이미지 파일 경로 설정
image_folder = '/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result_안양/'
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

# 파일명에서 datetime 추출하여 정렬
image_files.sort(key=lambda x: x.split('_')[-1].replace('.png', ''))  # '_dt' 부분에서 datetime 문자열로 정렬

# 이미지 크기 설정 (원하는 크기로 변경)
desired_size = (800, 800)  # 예: 800x800 픽셀

# 이미지 리스트의 각 이미지를 원하는 크기로 조정
resized_images = []
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)  # 전체 경로 생성
    image = Image.open(img_path)  # PIL로 이미지 열기
    resized_image = image.resize(desired_size, Image.LANCZOS)  # 이미지 크기 조정
    resized_images.append(resized_image)

# GIF 생성
gif_path = '/content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result_안양/jefestival_od.gif'
imageio.mimsave(gif_path, resized_images, fps=3, loop=True)
print(f"GIF 파일이 생성되었습니다: {gif_path}")

GIF 파일이 생성되었습니다: /content/drive/MyDrive/Colab Notebooks/빅콘(2024)/od_visual/result_안양/jefestival_od.gif
